In [39]:
import joblib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Load model and encoders
team_encoder = joblib.load("../model/team_encoder.pkl")
opponent_encoder = joblib.load("../model/opponent_encoder.pkl")
scaler = joblib.load("../model/scaler.pkl")
model = joblib.load("../model/nba_score_model.pkl")

team_names = list(team_encoder.classes_)

# Title with new background color
display(HTML("""
    <div style='background-color: #0066cc; padding: 20px; border-radius: 10px;'>
        <h2 style='color: #ffffff; font-family: "Arial", sans-serif; font-size: 32px; text-align: center;'>
            🏀 NBA Score Prediction
        </h2>
    </div>
"""))

# Team selectors with updated color and font
team_widget = widgets.Dropdown(
    options=team_names,
    value="Lakers",  # Default value
    description="Your Team:",
    style={'description_width': 'initial', 'font_size': '16px', 'font_family': 'Arial, sans-serif'},
    layout=widgets.Layout(width='90%', margin='10px 0px', padding='5px')
)

opponent_widget = widgets.Dropdown(
    options=team_names,
    value="Heat",  # Default value
    description="Opponent Team:",
    style={'description_width': 'initial', 'font_size': '16px', 'font_family': 'Arial, sans-serif'},
    layout=widgets.Layout(width='90%', margin='10px 0px', padding='5px')
)

# Function to create styled text inputs with updated font and color
def create_input(description, placeholder):
    return widgets.Text(
        description=description,
        placeholder=placeholder,
        style={'description_width': '80px', 'font_size': '16px', 'font_family': 'Arial, sans-serif'},
        layout=widgets.Layout(width='95%', margin='10px 0px', padding='5px')
    )

# Create input fields with updated style
def create_stat_inputs(fg="45", ft="75", fg3="35", ast="25", reb="40"):
    return {
        "fg_pct": create_input("FG%:", fg),
        "ft_pct": create_input("FT%:", ft),
        "fg3_pct": create_input("3PT%:", fg3),
        "ast": create_input("AST:", ast),
        "reb": create_input("REB:", reb)
    }

# Differentiating initial stats for team and opponent
team_stats = create_stat_inputs(fg="45", ft="75", fg3="35", ast="25", reb="40")
opp_stats = create_stat_inputs(fg="47", ft="72", fg3="38", ast="28", reb="42")

# Function to create sliders with updated design
def create_slider(description, value, min_val, max_val):
    return widgets.FloatSlider(
        value=value, min=min_val, max=max_val, step=0.01,
        description=description, readout_format='.2f',
        style={'description_width': '80px', 'font_size': '16px', 'font_family': 'Arial, sans-serif'},
        layout=widgets.Layout(width='95%', height='60px', margin='10px 0px', padding='5px'),
        readout_style={'color': 'black', 'font-size': '16px'}
    )

# Function to create integer sliders
def create_int_slider(description, value, min_val, max_val):
    return widgets.IntSlider(
        value=value, min=min_val, max=max_val, step=1,
        description=description, readout_format='d',
        style={'description_width': '80px', 'font_size': '16px', 'font_family': 'Arial, sans-serif'},
        layout=widgets.Layout(width='95%', height='60px', margin='10px 0px', padding='5px'),
        readout_style={'color': 'black', 'font-size': '16px'}
    )

# Create sliders for stats with improved visual design
fg_pct_slider_team = create_slider('.FG%', 0.45, 0.3, 0.6)
ft_pct_slider_team = create_slider('.FT%', 0.75, 0.5, 1.0)
fg3_pct_slider_team = create_slider('.3PT%', 0.35, 0.2, 0.5)
ast_slider_team = create_int_slider('AST', 25, 5, 40)
reb_slider_team = create_int_slider('REB', 40, 20, 60)

# Create sliders for opponent stats with updated design
fg_pct_slider_opp = create_slider('.FG%', 0.47, 0.3, 0.6)
ft_pct_slider_opp = create_slider('.FT%', 0.72, 0.5, 1.0)
fg3_pct_slider_opp = create_slider('.3PT%', 0.38, 0.2, 0.5)
ast_slider_opp = create_int_slider('AST', 28, 5, 40)
reb_slider_opp = create_int_slider('REB', 42, 20, 60)

# Prediction button with updated design
predict_button = widgets.Button(
    description="🎯 Predict Score",
    button_style="success",
    layout=widgets.Layout(width='60%', height='40px'),
    style={'font_weight': 'bold', 'font_size': '18px'}
)

output = widgets.Output()

# Prediction logic
def predict_score(b):
    with output:
        clear_output()
        try:
            # Encode teams
            team_encoded = team_encoder.transform([team_widget.value])[0]
            opponent_encoded = opponent_encoder.transform([opponent_widget.value])[0]

            # Prepare stats values for prediction
            team_values = [
                team_encoded, opponent_encoded,
                fg_pct_slider_team.value, ft_pct_slider_team.value, fg3_pct_slider_team.value,
                ast_slider_team.value, reb_slider_team.value
            ]
            opponent_values = [
                opponent_encoded, team_encoded,
                fg_pct_slider_opp.value, ft_pct_slider_opp.value, fg3_pct_slider_opp.value,
                ast_slider_opp.value, reb_slider_opp.value
            ]

            # Prepare for model
            team_scaled = scaler.transform([team_values])
            opponent_scaled = scaler.transform([opponent_values])

            # Make prediction
            team_score = model.predict(team_scaled)[0]
            opponent_score = model.predict(opponent_scaled)[0]

            # Output result
            display(HTML(f"""
                <div style='background-color: #0066cc; padding: 20px; border-radius: 10px;
                             color: #ffffff; font-family: Arial; font-size: 20px;'>
                    <b>Predicted Score:</b><br><br>
                    🔵 <b>{team_widget.value}</b>: {int(team_score)} points<br>
                    🔴 <b>{opponent_widget.value}</b>: {int(opponent_score)} points
                </div>
            """))

        except Exception as e:
            display(HTML(f"<span style='color: red;'>⚠️ Error: {str(e)}</span>"))

# Bind event to the button
predict_button.on_click(predict_score)

# Layout with updated design
dropdowns = widgets.HBox([team_widget, opponent_widget])

team_box = widgets.VBox([
    widgets.HTML("<h4 style='color:#ffffff; font-family: Arial, sans-serif;'>📊 Your Team Stats</h4>"),
    fg_pct_slider_team, ft_pct_slider_team, fg3_pct_slider_team, ast_slider_team, reb_slider_team
])

# Add a horizontal line between team stats and opponent stats
divider = widgets.HTML("<hr style='border: 1px solid #ffffff; width: 90%; margin: 10px 0;'>")

opponent_box = widgets.VBox([
    widgets.HTML("<h4 style='color:#ffffff; font-family: Arial, sans-serif;'>📊 Opponent Stats</h4>"),
    fg_pct_slider_opp, ft_pct_slider_opp, fg3_pct_slider_opp, ast_slider_opp, reb_slider_opp
])

# Add a horizontal line between the teams' names and their stats
team_opponent_divider = widgets.HTML("<hr style='border: 1px solid #ffffff; width: 90%; margin: 10px 0;'>")

inputs_box = widgets.VBox([team_box, team_opponent_divider, opponent_box])

# Display everything inside a light blue panel
display(HTML("<div style='background-color: #0066cc; padding: 20px; border-radius: 10px;'>"))
display(dropdowns)
display(inputs_box)
display(predict_button)
display(output)
display(HTML("</div>"))

Button(button_style='success', description='🎯 Predict Score', layout=Layout(height='40px', width='60%'), style…

Output()